In [18]:
import numpy as np
import cv2
import imageio
import matplotlib.pyplot as plt
import math

In [22]:
class NeedleVisualization:

    def __init__(self):
        self.frameWidth = 440
        self.frameHeight = 440
        
        #ROI parameters
        self.rstart = 140 #previously 94
        self.rend = 348
        self. cstart = 195 #previously 166
        self. cend = 235 #previously 275

    def insert_frame(self, frame):
        self.frame = frame
        
        #Initial Preprocessing
        self.resized_frame = cv2.resize(self.frame, (self.frameWidth,self.frameHeight))
        self.resized_frame = cv2.cvtColor(self.resized_frame, cv2.COLOR_RGB2GRAY) 

    def line_creation(self,source_image, overlay):
        lines = cv2.HoughLinesP(source_image, rho=6, theta=np.pi / 2, threshold=160, lines=np.array([]), minLineLength=40, maxLineGap=4)

        overlay_image = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
        houghline = overlay_image.copy()

        if lines is not None:
            for line in lines:
                
                start_point = (line[0][0], line[0][1]) # represents the top left corner of image
                end_point = (line[0][2], line[0][3]) # represents the bottom right corner of image
                color = (0, 255, 0) # Green color in BGR
                thickness = 2 # Line thickness
                    
                cv2.line(houghline, start_point, end_point, color, thickness)
        
        return houghline
    
    def line_creation2(self, source_image, overlay):
    
        lines = cv2.HoughLinesP(source_image, rho=6, theta=np.pi / 2, threshold=160, lines=np.array([]), minLineLength=40, maxLineGap=4)
        overlay_image = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
        
        houghline = overlay_image.copy()
        houghcircle = overlay_image.copy()
        
        length_line_list = []

        if lines is not None:
            for line in lines:

                x1 = line[0][0]
                y1 = line[0][1]
                x2 = line[0][2]
                y2 = line[0][3]

                start_point = (x1, y1)
                end_point = (x2, y2)
                
                lengthOfLine = math.sqrt(abs(x2-x1)^2 + abs(y2-y1)^2)
                length_line_list.append(lengthOfLine)

            index_number = length_line_list.index(max(length_line_list))

            x1 = lines[index_number][0][0]
            y1 = lines[index_number][0][1]
            x2 = lines[index_number][0][2]
            y2 = lines[index_number][0][3]

            start_point = (x1, y1)
            end_point = (x2, y2)

            color = (0, 255, 0) # Green color in BGR
            thickness = 2 # Line thickness of 9 px
            radius = 5 #circle radius

            cv2.line(houghline, start_point, end_point, color, thickness)
            cv2.circle(houghcircle, end_point, radius, color, thickness)
        
        return houghline

    def circle_creation(self, source_image, overlay):
        lines = cv2.HoughLinesP(source_image, rho=6, theta=np.pi / 2, threshold=160, lines=np.array([]), minLineLength=40, maxLineGap=4)
        overlay_image = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
        houghcircle = overlay_image.copy()
        
        length_line_list = []

        if lines is not None:
            for line in lines:

                x1 = line[0][0]
                y1 = line[0][1]
                x2 = line[0][2]
                y2 = line[0][3]
                
                lengthOfLine = math.sqrt(abs(x2-x1)^2 + abs(y2-y1)^2)
                length_line_list.append(lengthOfLine)

            index_number = length_line_list.index(max(length_line_list))

            x1 = lines[index_number][0][0]
            y1 = lines[index_number][0][1]
            x2 = lines[index_number][0][2]
            y2 = lines[index_number][0][3]

            start_point = (x1, y1)

            color = (0, 255, 0) # Green color in BGR
            thickness = 2 # Line thickness of 9 px
            radius = 5 #circle radius

            cv2.circle(houghcircle, start_point, radius, color, thickness)
        
        return houghcircle

    def ROI_creation(self,source_image, row_start, row_end, col_start, col_end):
        ROI_frame = source_image[row_start:row_end, col_start:col_end] #old one was [94:348, 166:275]
        ROI_image = np.zeros_like(source_image)
        x = row_start 
        y = col_start 
        for i in range(0, row_end-row_start):
            for j in range(0, col_end-col_start):
                if ROI_frame[i][j] != 0:
                    ROI_image[x + i, y + j] = ROI_frame[i, j]
        return ROI_image

    
    def detect_needle_line(self):

        #Achieving desired region of interest within Raw Frame
        ##############################################################
        ROI_image = self.ROI_creation(self.resized_frame,self.rstart,self.rend,self.cstart,self.cend)
        ##############################################################

        #Applying Paper Algorithm Filters
        #############################################################
        # gabor_filter = cv2.getGaborKernel((6,6), sigma=0.5, theta=0, lambd=0.5, gamma=0.8, psi=0, ktype=cv2.CV_32F)
        gabor_filter = cv2.getGaborKernel((3,3), sigma=0.95, theta=0, lambd=5, gamma=0.8, psi=0, ktype=cv2.CV_32F)
        # gabor_filter = cv2.getGaborKernel((3,3), sigma=0.5, theta=0, lambd=30, gamma=0.8, psi=0, ktype=cv2.CV_32F)

        gabor_output = cv2.filter2D(ROI_image, -1, gabor_filter)

        #Binarized image is divided into grids for needle axis localization.
        # - Median filter
        median_filter = cv2.medianBlur(gabor_output, 7)
        # - automatic thresholding
        threshold = cv2.threshold(median_filter, 250, 255, cv2.THRESH_BINARY)[1]
        # - morphological operations
        element = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        eroded = cv2.erode(threshold, element)
        dilated = cv2.dilate(eroded, element)
        #############################################################
            
        #Hough Line Transforms
        #############################################################
        houghline = self.line_creation2(dilated, self.resized_frame)
        #############################################################
        return houghline
    
    def detect_needle_tip(self):

        #Achieving desired region of interest within Raw Frame
        ##############################################################
        ROI_image = self.ROI_creation(self.resized_frame,self.rstart,self.rend,self.cstart,self.cend)
        ##############################################################

        #Applying Paper Algorithm Filters
        #############################################################
        # gabor_filter = cv2.getGaborKernel((6,6), sigma=0.5, theta=0, lambd=0.5, gamma=0.8, psi=0, ktype=cv2.CV_32F)
        gabor_filter = cv2.getGaborKernel((3,3), sigma=0.95, theta=0, lambd=5, gamma=0.8, psi=0, ktype=cv2.CV_32F)
        # gabor_filter = cv2.getGaborKernel((3,3), sigma=0.5, theta=0, lambd=30, gamma=0.8, psi=0, ktype=cv2.CV_32F)

        gabor_output = cv2.filter2D(ROI_image, -1, gabor_filter)

        #Binarized image is divided into grids for needle axis localization.
        # - Median filter
        median_filter = cv2.medianBlur(gabor_output, 7)
        # - automatic thresholding
        threshold = cv2.threshold(median_filter, 250, 255, cv2.THRESH_BINARY)[1]
        # - morphological operations
        element = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        eroded = cv2.erode(threshold, element)
        dilated = cv2.dilate(eroded, element)
        #############################################################
            
        #Hough Line Transforms
        #############################################################
        houghcircle = self.circle_creation(dilated, self.resized_frame)
        #############################################################
        return houghcircle
    
    def show_needle_viz(self, houghline):
        resized_frame = cv2.cvtColor(self.resized_frame, cv2.COLOR_GRAY2BGR)
        overlay = cv2.cvtColor(houghline, cv2.COLOR_RGB2BGR)
        stack = np.hstack((resized_frame, overlay))
        cv2.imshow("stacked", stack)
        return stack
    
    # def show_debug_frames():
    

In [26]:
#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'
NeedleViz_oilAndLatex = 'Data/edited data/oil and latex/capture_5_2022-11-12T16-56-03.mp4'
NeedleViz_gelAndLatex = 'Data/edited data/ultrasound gel and latex/capture_4_2022-11-12T17-33-19.mp4'
NeedleViz_clarius1 = 'Data/edited data/clarius_FinalPrototype_needlejustWater.mp4'
NeedleViz_clarius2 = 'Data/edited data/clarius_FinalPrototype_needlejustWater2.mp4'
NeedleViz_clarius3 = 'Data/edited data/clarius_FinalPrototype_needleWithSolid.mp4'
NeedleViz_clarius4 = 'Data/edited data/clarius_FinalPrototype_needleWithSolid2.mp4'
NeedleViz_clarius5 = 'Data/edited data/clarius_FinalPrototype_needleWithSolid3.mp4'

#control playback speed
frame_rate = 30

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_clarius2)

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

size = (frameWidth, frameHeight)

#Preparing to create output videos
image_lst = []
save = True

#Initialize class
needleviz = NeedleVisualization()

if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval:

        needleviz.insert_frame(frame)
        houghline = needleviz.detect_needle_tip()
        stack = needleviz.show_needle_viz(houghline)
        
        if save:
            image_lst.append(stack)

        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

In [27]:
#Saving Video as GIF
imageio.mimsave('Outputs/V3_3_video.gif', image_lst, fps=30)